In [ ]:
!pip install faiss-cpu

In [ ]:
# ================================
# STEP 1: INSTALL REQUIREMENTS
# ================================
print("📦 Installing packages...")

!pip install pandas numpy scikit-learn xgboost lightgbm catboost \
    fuzzywuzzy python-Levenshtein \
    transformers sentence-transformers

print("✅ All packages installed!")

import pandas as pd, numpy as np, re, os
from fuzzywuzzy import fuzz
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sentence_transformers import SentenceTransformer
import faiss
from collections import Counter, defaultdict
import warnings
warnings.filterwarnings("ignore")

📦 Installing packages...
✅ All packages installed!


In [ ]:
# ================================
# STEP 2: TURKISH NLP + GEO DB
# ================================

    def __init__(self):
        # All 81 Turkish provinces with major districts
        self.turkish_administrative_units = {
            'adana': ['seyhan', 'yuregir', 'cukurova', 'sariçam', 'karaisali', 'karatas'],
            'adiyaman': ['merkez', 'besni', 'çelikhan', 'gerger', 'golbasi', 'kahta', 'samsat', 'sincik', 'tut'],
            'afyonkarahisar': ['merkez', 'sandikli', 'dinar', 'bolvadin', 'çay', 'dazkiri', 'emirdağ'],
            'agri': ['merkez', 'dogubayazit', 'patnos', 'tutak', 'diyadin', 'eleskirt', 'hamur', 'taslicay'],
            'amasya': ['merkez', 'merzifon', 'suluova', 'tasova', 'gokdere', 'hamamözü'],
            'ankara': ['cankaya', 'kecioren', 'yenimahalle', 'mamak', 'altindag', 'sincan', 'etimesgut',
                      'golbasi', 'polatli', 'pursaklar', 'akyurt', 'ayaş', 'bala', 'beypazari'],
            'antalya': ['muratpasa', 'kepez', 'konyaalti', 'alanya', 'manavgat', 'serik', 'kas', 'kemer',
                       'elmali', 'finike', 'gazipaşa', 'demre', 'kumluca', 'akseki'],
            'artvin': ['merkez', 'hopa', 'borçka', 'arhavi', 'yusufeli', 'şavşat', 'ardanuç', 'murgul'],
            'aydin': ['efeler', 'nazilli', 'söke', 'kuşadasi', 'didim', 'çine', 'bozdogan', 'germencik'],
            'balikesir': ['karesi', 'altieylul', 'bandirma', 'edremit', 'ayvalik', 'burhaniye', 'erdek'],
            'bartin': ['merkez', 'amasra', 'kurucasile', 'ulus'],
            'batman': ['merkez', 'kozluk', 'besiri', 'gercus', 'hasankeyf', 'sason'],
            'bayburt': ['merkez', 'aydıntepe', 'demirozu'],
            'bilecik': ['merkez', 'bozuyuk', 'sogut', 'osmaneli', 'golpazari', 'inhisar', 'pazaryeri', 'yenipazar'],
            'bingol': ['merkez', 'genc', 'karliova', 'solhan', 'adakli', 'kigi', 'yayladere', 'yedisu'],
            'bitlis': ['merkez', 'tatvan', 'güroymak', 'hizan', 'mutki', 'adilcevaz', 'ahlat'],
            'bolu': ['merkez', 'düzce', 'goynuk', 'mudurnu', 'mengen', 'gerede', 'kibriscik', 'seben'],
            'burdur': ['merkez', 'bucak', 'golhisar', 'yesilova', 'karamanli', 'aglasun', 'altinyayla'],
            'bursa': ['osmangazi', 'nilufer', 'yildirim', 'mudanya', 'gemlik', 'inegol', 'orhaneli',
                     'buyukorhan', 'harmancik', 'iznik', 'karacabey', 'keles', 'kestel', 'mustafakemalpasa'],
            'canakkale': ['merkez', 'gelibolu', 'biga', 'çanakkale', 'ayvacik', 'bayramiç', 'bozcaada'],
            'cankiri': ['merkez', 'çerkes', 'ilgaz', 'kurşunlu', 'orta', 'atkaracalar', 'bayramören'],
            'corum': ['merkez', 'osmancik', 'iskilip', 'kargi', 'dodurga', 'alaca', 'bayat', 'bogazkale'],
            'denizli': ['pamukkale', 'merkezefendi', 'honaz', 'tavas', 'çal', 'acipayam', 'buldan', 'çameli'],
            'diyarbakir': ['baglar', 'kayapinar', 'sur', 'yenisehir', 'bismil', 'çermik', 'çinar', 'dicle'],
            'edirne': ['merkez', 'kesan', 'uzunkopru', 'ipsala', 'havsa', 'enez', 'lalapaşa', 'meriç', 'süloglu'],
            'elazig': ['merkez', 'karakocan', 'keban', 'palu', 'sivrice', 'ağın', 'alacakaya', 'arıcak', 'baskil'],
            'erzincan': ['merkez', 'üzümlü', 'refahiye', 'tercan', 'kemah', 'çayırlı', 'iliç', 'kemaliye', 'otlukbeli'],
            'erzurum': ['yakutiye', 'palandoken', 'aziziye', 'hinis', 'pasinler', 'aşkale', 'çat', 'horasan'],
            'eskisehir': ['tepebasi', 'odunpazari', 'sivrihisar', 'çifteler', 'alpu', 'beylikova', 'çifteler'],
            'gaziantep': ['şahinbey', 'şehitkamil', 'nizip', 'islahiye', 'nurdagi', 'araban', 'karkamış'],
            'giresun': ['merkez', 'bulancak', 'espiye', 'gorele', 'tirebolu', 'alucra', 'çamoluk', 'çanakçı'],
            'gümüşhane': ['merkez', 'kelkit', 'şiran', 'torul', 'kose', 'kürtün'],
            'hakkari': ['merkez', 'yüksekova', 'şemdinli', 'çukurca'],
            'hatay': ['antakya', 'defne', 'arsuz', 'dortyol', 'iskenderun', 'kirikhan', 'payas', 'reyhanli', 'samandağ'],
            'igdir': ['merkez', 'tuzluca', 'karakoyunlu', 'aralik'],
            'isparta': ['merkez', 'yalvaç', 'keçiborlu', 'şarkikaraağaç', 'atabey', 'eğirdir', 'gelendost'],
            'istanbul': ['fatih', 'beyoglu', 'uskudar', 'kadikoy', 'besiktas', 'sisli', 'bakirkoy', 'zeytinburnu',
                        'esenler', 'gaziosmanpasa', 'kagithane', 'sariyer', 'maltepe', 'pendik', 'ümraniye',
                        'beykoz', 'beylikdüzü', 'esenyurt', 'avcılar', 'küçükçekmece', 'büyükçekmece',
                        'bahçelievler', 'bağcılar', 'güngören', 'sultangazi', 'arnavutköy', 'başakşehir',
                        'bayrampaşa', 'beşiktaş', 'eyüpsultan', 'kağıthane', 'kartal', 'silivri', 'sultanbeyli'],
            'izmir': ['konak', 'bornova', 'çigli', 'karşıyaka', 'bayrakli', 'gaziemir', 'balçova', 'narlidere',
                     'buca', 'menderes', 'torbalı', 'menemen', 'foça', 'urla', 'çeşme', 'ödemiş', 'tire',
                     'bergama', 'dikili', 'karaburun', 'kınık', 'kiraz', 'selçuk', 'aliağa'],
            'kahramanmaras': ['onikişubat', 'dulkadiroğlu', 'pazarcık', 'elbistan', 'afşin', 'andırın'],
            'karabük': ['merkez', 'safranbolu', 'yenice', 'eskipazar', 'ovacık', 'eflani'],
            'karaman': ['merkez', 'ermenek', 'kazımkarabekir', 'başyayla', 'ayrancı', 'sarıveliler'],
            'kars': ['merkez', 'kagizman', 'ardahan', 'arpaçay', 'selim', 'akyaka', 'digor', 'susuz'],
            'kastamonu': ['merkez', 'taşköprü', 'sinop', 'boyabat', 'çankırı', 'abana', 'ağlı', 'araç'],
            'kayseri': ['melikgazi', 'kocasinan', 'talas', 'develi', 'yahyalı', 'bünyan', 'felahiye', 'hacılar'],
            'kırıkkale': ['merkez', 'yahşihan', 'keskin', 'sulakyurt', 'bahşılı', 'balışeyh', 'çelebi', 'delice'],
            'kirklareli': ['merkez', 'babaeski', 'luleburgaz', 'pınarhisar', 'demirköy', 'kofçaz', 'pehlivanköy', 'vize'],
            'kirsehir': ['merkez', 'kaman', 'mucur', 'çiçekdağ', 'akçakent', 'akpınar', 'boztepe'],
            'kilis': ['merkez', 'musabeyli', 'polateli', 'elbeyli'],
            'kocaeli': ['izmit', 'gebze', 'darıca', 'körfez', 'gölcük', 'kandıra', 'başiskele', 'çayırova', 'derince', 'dilovası', 'kartepe'],
            'konya': ['selçuklu', 'meram', 'karatay', 'eregli', 'akşehir', 'beyşehir', 'çumra', 'ilgın', 'kulu'],
            'kütahya': ['merkez', 'tavşanlı', 'gediz', 'simav', 'emet', 'altıntaş', 'aslanapa', 'çavdarhisar'],
            'malatya': ['yeşilyurt', 'battalgazi', 'akçadağ', 'darende', 'doğanşehir', 'hekimhan', 'kuluncak', 'pütürge'],
            'manisa': ['yunusemre', 'şehzadeler', 'turgutlu', 'akhisar', 'salihli', 'alaşehir', 'demirci', 'gördes'],
            'mardin': ['artuklu', 'midyat', 'kızıltepe', 'nusaybin', 'ömerli', 'dargeçit', 'derik', 'mazıdağı'],
            'mersin': ['yenişehir', 'mezitli', 'toroslar', 'akdeniz', 'tarsus', 'erdemli', 'anamur', 'aydıncık'],
            'muğla': ['menteşe', 'bodrum', 'marmaris', 'fethiye', 'milas', 'ortaca', 'çine', 'datça', 'köyceğiz'],
            'muş': ['merkez', 'bulanık', 'malazgirt', 'varto', 'hasköy', 'korkut'],
            'nevşehir': ['merkez', 'ürgüp', 'avanos', 'göreme', 'derinkuyu', 'acıgöl', 'çat', 'hacıbektaş'],
            'niğde': ['merkez', 'bor', 'ulukışla', 'çamardı', 'altunhisar', 'çiftlik'],
            'ordu': ['altınordu', 'ünye', 'fatsa', 'perşembe', 'çaybaşı', 'akkuş', 'aybasti', 'çamaş'],
            'osmaniye': ['merkez', 'kadirli', 'düziçi', 'bahçe', 'hasanbeyli', 'sumbas', 'toprakkale'],
            'rize': ['merkez', 'çayeli', 'ardeşen', 'pazar', 'fındıklı', 'güneysu', 'hemşin', 'ikizdere'],
            'sakarya': ['adapazarı', 'serdivan', 'akyazı', 'karasu', 'hendek', 'arifiye', 'erenler', 'ferizli'],
            'samsun': ['ilkadım', 'canik', 'atakum', 'bafra', 'çarşamba', 'terme', 'alaçam', 'asarcık'],
            'şanlıurfa': ['eyyübiye', 'haliliye', 'karaköprü', 'viranşehir', 'suruç', 'akçakale', 'birecik'],
            'siirt': ['merkez', 'kurtalan', 'şirvan', 'baykan', 'pervari', 'aydınlar', 'eruh'],
            'sinop': ['merkez', 'boyabat', 'ayancık', 'türkeli', 'erfelek', 'durağan', 'gerze'],
            'şırnak': ['merkez', 'cizre', 'silopi', 'idil', 'güçlükonak', 'beytüşşebap', 'uludere'],
            'sivas': ['merkez', 'şarkışla', 'yıldızeli', 'suşehri', 'divriği', 'gemerek', 'gürün', 'hafik'],
            'tekirdağ': ['süleymanpaşa', 'çorlu', 'çerkezköy', 'hayrabolu', 'kapaklı', 'malkara', 'marmaraereğlisi', 'muratlı', 'saray', 'şarköy'],
            'tokat': ['merkez', 'turhal', 'erbaa', 'niksar', 'reşadiye', 'almus', 'artova', 'başçiftlik'],
            'trabzon': ['ortahisar', 'akçaabat', 'vakfıkebir', 'of', 'araklı', 'arsin', 'beşikdüzü', 'çaykara'],
            'tunceli': ['merkez', 'çemişgezek', 'hozat', 'mazgirt', 'ovacık', 'pertek', 'pülümür'],
            'uşak': ['merkez', 'banaz', 'eşme', 'karahallı', 'sivaslı', 'ulubey'],
            'van': ['ipekyolu', 'tuşba', 'edremit', 'gevaş', 'muradiye', 'bahçesaray', 'başkale', 'çaldıran'],
            'yalova': ['merkez', 'çiftlikköy', 'altınova', 'armutlu', 'çınarcık', 'termal'],
            'yozgat': ['merkez', 'sorgun', 'boğazlıyan', 'yerköy', 'şefaatli', 'akdağmadeni', 'aydıncık'],
            'zonguldak': ['merkez', 'ereğli', 'çaycuma', 'devrek', 'gökçebey', 'alaplı', 'kilimli']
        }

        # City abbreviations and common variations
        self.city_variations = {
            'ist': 'istanbul', 'izm': 'izmir', 'ank': 'ankara', 'bur': 'bursa',
            'ada': 'adana', 'ant': 'antalya', 'mer': 'mersin', 'gaz': 'gaziantep',
            'kon': 'konya', 'kay': 'kayseri', 'den': 'denizli', 'man': 'manisa',
            'tra': 'trabzon', 'sam': 'samsun', 'esk': 'eskisehir', 'mal': 'malatya',
            # Common misspellings
            'uskuar': 'uskudar', 'kadkoy': 'kadikoy', 'besiktas': 'besiktas',
            'galtsaray': 'galatasaray', 'taksm': 'taksim', 'sisli': 'sisli',
            'bagcilar': 'bagcilar', 'umranye': 'umraniye', 'pendik': 'pendik'
        }

        # Postal code prefixes (first 2 digits)
        self.postal_prefixes = {
            '01': 'adana', '02': 'adiyaman', '03': 'afyonkarahisar', '04': 'agri',
            '05': 'amasya', '06': 'ankara', '07': 'antalya', '08': 'artvin',
            '09': 'aydin', '10': 'balikesir', '11': 'bilecik', '12': 'bingol',
            '13': 'bitlis', '14': 'bolu', '15': 'burdur', '16': 'bursa',
            '17': 'canakkale', '18': 'cankiri', '19': 'corum', '20': 'denizli',
            '21': 'diyarbakir', '22': 'edirne', '23': 'elazig', '24': 'erzincan',
            '25': 'erzurum', '26': 'eskisehir', '27': 'gaziantep', '28': 'giresun',
            '29': 'gümüşhane', '30': 'hakkari', '31': 'hatay', '32': 'isparta',
            '33': 'mersin', '34': 'istanbul', '35': 'izmir', '36': 'kars',
            '37': 'kastamonu', '38': 'kayseri', '39': 'kirklareli', '40': 'kirsehir',
            '41': 'kocaeli', '42': 'konya', '43': 'kütahya', '44': 'malatya',
            '45': 'manisa', '47': 'mardin', '48': 'mugla', '49': 'muş',
            '50': 'nevşehir', '51': 'niğde', '52': 'ordu', '53': 'rize',
            '54': 'sakarya', '55': 'samsun', '56': 'siirt', '57': 'sinop',
            '58': 'sivas', '59': 'tekirdağ', '60': 'tokat', '61': 'trabzon',
            '62': 'tunceli', '63': 'şanlıurfa', '64': 'uşak', '65': 'van',
            '66': 'yozgat', '67': 'zonguldak', '68': 'aksaray', '69': 'bayburt',
            '70': 'karaman', '71': 'kırıkkale', '72': 'batman', '73': 'şırnak',
            '74': 'bartin', '75': 'ardahan', '76': 'igdir', '77': 'yalova',
            '78': 'karabük', '79': 'kilis', '80': 'osmaniye', '81': 'düzce'
        }
    def find_province(self,address):
        addr=address.lower()
        for p,vars in self.provinces.items():
            if any(v in addr for v in vars): return p
        return None
    def find_district(self,address):
        addr=address.lower()
        for d,vars in self.districts.items():
            if any(v in addr for v in vars): return d
        return None

In [ ]:
# ================================
# STEP 3: NORMALIZER
# ================================
class AddressNormalizer:
    def normalize(self,address:str):
        if pd.isna(address): return ""
        address=str(address).lower()
        address=re.sub(r"[^\w\s]", " ", address)
        address=re.sub(r"\s+"," ",address)
        abbr = {r"\bmah\b":"mahalle", r"\bsok\b":"sokak", r"\bcd\b":"cadde",
                r"\bapt\b":"apartman", r"\bno\b":"numara", r"\bd\b":"daire"}
        for k,v in abbr.items(): address=re.sub(k,v,address)
        return address.strip()

In [ ]:
# ================================
# STEP 3: NORMALIZER
# ================================
class AddressNormalizer:
    def normalize(self,address:str):
        if pd.isna(address): return ""
        address=str(address).lower()
        address=re.sub(r"[^\w\s]", " ", address)
        address=re.sub(r"\s+"," ",address)
        abbr = {r"\bmah\b":"mahalle", r"\bsok\b":"sokak", r"\bcd\b":"cadde",
                r"\bapt\b":"apartman", r"\bno\b":"numara", r"\bd\b":"daire"}
        for k,v in abbr.items(): address=re.sub(k,v,address)
        return address.strip()

In [ ]:
# ================================
# STEP 4: FEATURE EXTRACTOR
#  - TF-IDF + SVD
#  - BERT Embeddings
#  - Geographic one-hot
# ================================

class FeatureExtractor:
    def __init__(self):
        self.tfidf = TfidfVectorizer(max_features=4000,ngram_range=(1,4),analyzer="char_wb")
        self.svd = TruncatedSVD(n_components=150,random_state=42)
        self.scaler = StandardScaler()
        # Türkçe BERT modeli (Sentence-BERT tabanlı)
        self.bert = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
        self.geo = TurkeyLocationDatabase()
    def fit_transform(self,addresses):
        norm=[AddressNormalizer().normalize(a) for a in addresses]
        tfidf_mat=self.tfidf.fit_transform(norm)
        tfidf_svd=self.svd.fit_transform(tfidf_mat)
        bert_emb=self.bert.encode(norm,show_progress_bar=True)
        geo_feats=[self._geo(addr) for addr in norm]
        feats=np.hstack([tfidf_svd,bert_emb,geo_feats])
        return self.scaler.fit_transform(feats)
    def transform(self,addresses):
        norm=[AddressNormalizer().normalize(a) for a in addresses]
        tfidf_mat=self.tfidf.transform(norm)
        tfidf_svd=self.svd.transform(tfidf_mat)
        bert_emb=self.bert.encode(norm,show_progress_bar=True)
        geo_feats=[self._geo(addr) for addr in norm]
        feats=np.hstack([tfidf_svd,bert_emb,geo_feats])
        return self.scaler.transform(feats)
    def _geo(self,address):
        p=self.geo.find_province(address); d=self.geo.find_district(address)
        provinces=["istanbul","ankara","izmir","bursa","antalya","manisa","mugla","aydin","denizli"]
        districts=["bornova","konak","karşıyaka","bayraklı","buca","fatih","kadıköy"]
        return [1 if p==pp else 0 for pp in provinces] + [1 if d==dd else 0 for dd in districts]

In [ ]:
# ================================
# STEP 5: ENSEMBLE MODEL
# ================================
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import lightgbm as lgb
from catboost import CatBoostClassifier

class EnsemblePredictor:
    def __init__(self):
        self.models={
            "rf":RandomForestClassifier(n_estimators=200,max_depth=20,n_jobs=-1,random_state=42),
            "xgb":XGBClassifier(n_estimators=200,max_depth=10,learning_rate=0.05,eval_metric="mlogloss"),
            "lgb":lgb.LGBMClassifier(n_estimators=200,max_depth=10,learning_rate=0.05),
            "cat":CatBoostClassifier(iterations=200,depth=8,learning_rate=0.05,verbose=0)
        }
    def fit(self,X,y):
        for name,m in self.models.items(): m.fit(X,y)
    def predict(self,X):
        preds={}
        for n,m in self.models.items(): preds[n]=m.predict(X)
        weights={"rf":0.2,"xgb":0.3,"lgb":0.3,"cat":0.2}
        final=[]
        for i in range(len(X)):
            votes=defaultdict(float)
            for mdl,p in preds.items(): votes[p[i]]+=weights[mdl]
            final.append(max(votes.items(),key=lambda x:x[1])[0])
        return np.array(final)

In [ ]:
# ================================
# STEP 6: ADVANCED DEDUP (FAISS)
# ================================
class Deduplicator:
    def __init__(self,emb_dim=384):
        self.index=faiss.IndexFlatIP(emb_dim) # cosine similarity
        self.embeddings=None
        self.ids=[]
    def build_index(self,addresses,ids):
        bert=SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
        norm=[AddressNormalizer().normalize(a) for a in addresses]
        self.embeddings=bert.encode(norm,normalize_embeddings=True)
        self.index.add(self.embeddings)
        self.ids=ids
    def find_duplicates(self,query_addresses,th=0.90):
        bert=SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
        norm=[AddressNormalizer().normalize(a) for a in query_addresses]
        q_emb=bert.encode(norm,normalize_embeddings=True)
        sims,I=self.index.search(q_emb,2) # top-2
        dup_map={}
        for i,(sim,idx) in enumerate(zip(sims,I)):
            if sim[1]>th: dup_map[i]=self.ids[idx[1]]
        return dup_map

In [ ]:
# ================================
# STEP 7: PIPELINE
# ================================

class AdvancedHybridResolver:
    def __init__(self):
        self.fe=FeatureExtractor()
        self.ensemble=EnsemblePredictor()
        self.dedup=Deduplicator()
    def fit(self,X,y,ids):
        feats=self.fe.fit_transform(X)
        self.ensemble.fit(feats,y)
        self.dedup.build_index(X,ids)
    def predict(self,X):
        feats=self.fe.transform(X)
        preds=self.ensemble.predict(feats)
        dup_map=self.dedup.find_duplicates(X)
        for i in dup_map: preds[i]=preds[dup_map[i]]  # duplicate correction
        return preds

In [ ]:
# ================================
# STEP 8: EXECUTION (SAMPLE)
# ================================
def run_solution():
    train=pd.read_csv("train.csv")
    test=pd.read_csv("test.csv")
    resolver=AdvancedHybridResolver()
    resolver.fit(train["address"].values,train["label"].values,train["id"].values)
    preds=resolver.predict(test["address"].values)
    sub=pd.DataFrame({"id":test["id"],"label":preds})
    sub.to_csv("hybrid_submission.csv",index=False)
    print("✅ Submission saved hybrid_submission.csv")
    return sub

# Run
# submission=run_solution()

In [ ]:
if __name__ == "__main__":
    print("🚀 Teknofest Hybrid Address Resolver Başladı")

    # Veri yükle
    try:
        train = pd.read_csv("train.csv")
        test = pd.read_csv("test.csv")
    except FileNotFoundError:
        print("❌ train.csv ve test.csv dosyalarını yüklemen gerekiyor!")
        from google.colab import files
        print("📤 Lütfen train.csv ve test.csv dosyalarını yükleyin:")
        uploaded = files.upload()
        for k in uploaded.keys():
            if "train" in k: train=pd.read_csv(k)
            if "test" in k: test=pd.read_csv(k)

    # Yoksa ID sütunu ekleyelim
    if "id" not in train.columns:
        train["id"] = np.arange(len(train))
    if "id" not in test.columns:
        test["id"] = np.arange(len(test))

    # Model oluştur
    resolver = AdvancedHybridResolver()

    # Eğit
    print("🎯 Model eğitiliyor...")
    resolver.fit(train["address"].values, train["label"].values, train["id"].values)

    # Tahmin
    print("🔮 Test verisi tahmin ediliyor...")
    predictions = resolver.predict(test["address"].values)

    # Submission kaydet
    submission = pd.DataFrame({"id": test["id"], "label": predictions})
    submission.to_csv("hybrid_submission.csv", index=False)
    print("✅ hybrid_submission.csv dosyası kaydedildi")

    # İlk satırlara bakalım
    print(submission.head())

🚀 Teknofest Hybrid Address Resolver Başladı
🎯 Model eğitiliyor...


Batches:   0%|          | 0/26508 [00:00<?, ?it/s]